array([[5, 4, 0, 1, 2, 1],
       [4, 0, 3, 1, 1, 2],
       [0, 5, 5, 0, 3, 3],
       [2, 0, 1, 4, 5, 4],
       [2, 2, 2, 0, 4, 0],
       [1, 2, 1, 0, 5, 4]], dtype=uint32)

In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('movie_baseline.csv', index_col=0)
data.values.astype('I')
data_view = data.style.highlight_null("teal")
data_view

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
u1,5.000000,4.000000,nan,1.000000,2,1.000000
u2,4.000000,nan,3.000000,1.000000,1,2.000000
u3,nan,5.000000,5.000000,nan,3,3.000000
u4,2.000000,nan,1.000000,4.000000,5,4.000000
u5,2.000000,2.000000,2.000000,nan,4,nan
u6,1.000000,2.000000,1.000000,nan,5,4.000000


1. 전체 평균을 구한다.

In [2]:
mu = round(data.sum().sum() / (data > 0).sum().sum() , 2)
mu

2.82

In [3]:
n_user, n_items = data.shape

2. 아이템 편향을 구함

![](bi.PNG)

In [4]:
item_biases = []

for i in range(n_items):
    users_reviewed = data.index[data.iloc[:, i] > 0].tolist()
    denom = 0
    for u in users_reviewed:
        denom += (data.loc[u, :][i] - mu).sum()
    bi = round(denom / len(users_reviewed), 2)
    item_biases.append(bi)

item_biases = pd.DataFrame.from_records([item_biases, ], index=['bias'], columns=data.columns.tolist())
item_biases

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
bias,-0.02,0.43,-0.42,-0.82,0.51,-0.02


3. 사용자 편향을 구함

![](bu.PNG)

In [5]:
user_biases = []

for u in range(n_user):
    item_rated = data.columns[data.iloc[u, :] > 0].tolist()
    denom = 0
    for i in item_rated:
        denom += (data.loc[:, i][u] - mu - item_biases[i]).sum()
    bu = round(denom / len(item_rated), 2)
    user_biases.append(bu)
    

user_biases = pd.DataFrame(user_biases, index=data.index, columns=['bias'])
user_biases

,bias
u1,-0.24
u2,-0.47
u3,1.06
u4,0.53
u5,-0.44
u6,-0.32


4. 합진다.

![](bui.PNG)

In [6]:
pred_rating = \
    mu + \
    np.repeat(user_biases.values, n_items, axis=1) + \
    np.repeat(item_biases.values, n_user, axis=0)

In [7]:
pred_rating = pd.DataFrame(pred_rating, index=data.index, columns=data.columns)
pred_rating

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
u1,2.56,3.01,2.16,1.76,3.09,2.56
u2,2.33,2.78,1.93,1.53,2.86,2.33
u3,3.86,4.31,3.46,3.06,4.39,3.86
u4,3.33,3.78,2.93,2.53,3.86,3.33
u5,2.36,2.81,1.96,1.56,2.89,2.36
u6,2.48,2.93,2.08,1.68,3.01,2.48


원본과 비교했을때

In [8]:
for idx in np.where(data.isna().values.flatten())[0]:
    i = idx // n_user
    j = idx % n_user
    data.iloc[i, j] = pred_rating.iloc[i, j]

In [9]:
data_view

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
u1,5.000000,4.000000,2.160000,1.000000,2,1.000000
u2,4.000000,2.780000,3.000000,1.000000,1,2.000000
u3,3.860000,5.000000,5.000000,3.060000,3,3.000000
u4,2.000000,3.780000,1.000000,4.000000,5,4.000000
u5,2.000000,2.000000,2.000000,1.560000,4,2.360000
u6,1.000000,2.000000,1.000000,1.680000,5,4.000000
